In [12]:
import os
import json
from sklearn.metrics import mean_absolute_error
import numpy as np

In [2]:
os.chdir("..")

## Importing the metadata

In [3]:
METADATA_PATH = "data/silver/metadata"

metadata = {}
for metadata_filename in os.listdir(METADATA_PATH):
    with open(f"{METADATA_PATH}/{metadata_filename}") as metadata_file:
        dish_metadata = json.load(metadata_file)
        metadata[dish_metadata["dish_id"]] = dish_metadata

## Importing the results

In [4]:
RESULTS_PATH = "results"

models_names = [
    "gemini",
    "gemini_split"
]

models_results = {}

for model_name in models_names:
    model_results_path = f"{RESULTS_PATH}/{model_name}"
    
    results = {}
    for result_filename in os.listdir(model_results_path):
        with open(f"{model_results_path}/{result_filename}") as result_file:
            result = json.load(result_file)
            results[result["dishId"]] = result
    
    models_results[model_name] = results

## Evaluating results

In [20]:
for model_name, model_results in models_results.items():
    Y_test_carb = []
    Y_pred_carb = []

    Y_test_calories = []
    Y_pred_calories = []

    processing_times = []

    for dish_id, dish_metadata in metadata.items():
        if dish_id in model_results:
            result = model_results.get(dish_id)

            Y_test_carb.append(dish_metadata["total_carb"])
            Y_pred_carb.append(result["totalCarbohydrates"])

            Y_test_calories.append(dish_metadata["total_calories"])
            Y_pred_calories.append(result["totalCalories"])

            processing_times.append(result["elapsedTime"])

    mae_carb = mean_absolute_error(Y_test_carb, Y_pred_carb)
    mae_calories = mean_absolute_error(Y_test_calories, Y_pred_calories)
    avg_time = np.mean(processing_times)

    print(f"{model_name.capitalize()}")
    print(f"-> MAE carbohydrates = {mae_carb:.1f} g")
    print(f"-> MAE calories = {mae_calories:.1f} kcal")
    print(f"-> Average processing time = {avg_time:.2f} s")
    print()

Gemini
-> MAE carbohydrates = 17.2 g
-> MAE calories = 113.6 kcal
-> Average processing time = 2.76 s

Gemini_split
-> MAE carbohydrates = 14.9 g
-> MAE calories = 113.5 kcal
-> Average processing time = 3.74 s

